In [1]:
import numpy as np
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile

In [2]:
#Read main data
data=pd.read_excel(r'D:\Personal\University\Master\Thesis\Aggregation-Weighting\data_values.xlsx',sheet_name='data_transpose')
rainfall_data=pd.read_excel(r'D:\Personal\University\Master\Thesis\Aggregation-Weighting\rainfall_data.xlsx',sheet_name='Sheet1')

# Read data necessary for normalization
ind_type=pd.read_excel(r'D:\Personal\University\Master\Thesis\Aggregation-Weighting\indicator_type_DEA.xlsx',sheet_name='Sheet1')

In [3]:
#defining normalization function for (dam capicity)/(surface rwr)
def damcap_normalize(ser1,ser2):
    result=ser1*0
    length=ser1.size
    for i in range(length):
        if ser1[i]>=ser2[i]:
            result[i]=(ser1.max()-ser1[i])/(ser1.max()-ser2[i])
        else:
            result[i]=(ser1[i]-ser1.min())/(ser2[i]-ser1.min())
        if result[i]<0.01:
            result[i]=0.01
    return result  

#defining aggregation function
def aggregate(df,a=0.5):
    size=len(df.columns)
    result=(df.product(axis=1)**(1/size))*a+df.mean(axis=1)*(1-a)
    return result
def addminagg(df,a=0.5):
    size=len(df.columns)
    result=(df.min(axis=1))*a+df.mean(axis=1)*(1-a)
    return result

In [4]:
#selecting rainfall data after 1366 (nearly 30 years)
rainfall_modified=rainfall_data.loc[(rainfall_data['syear']>1366) & (rainfall_data['syear']<1399)]

#making a dictionary of provinces and their respective data
pr=list(rainfall_modified.ostan.unique())
prdict={elem:pd.DataFrame() for elem in pr}
for key in prdict.keys():
    prdict[key]=rainfall_modified[:][rainfall_modified.ostan==key]
    
#making a dictionary consisting of province names as keys and annual precipitation as respective values
annualrain_dict={x:pd.DataFrame() for x in pr}
for key in annualrain_dict.keys():
    annualrain_dict[key]=prdict[key].groupby('syear').precnew.sum()

#calculating coefficients of variation
interannual_varicoef={x:pd.DataFrame() for x in pr}
for key in interannual_varicoef.keys():
    interannual_varicoef[key]=annualrain_dict[key].std()/annualrain_dict[key].mean()
monthly_varicoef={x:pd.DataFrame() for x in pr}
for key in monthly_varicoef.keys():
    monthly_varicoef[key]=prdict[key].precnew.std()/prdict[key].precnew.mean()
annualevap_varicoef={x:pd.DataFrame() for x in pr}

#converting monthly coefficient of variation to dataframe and sorting it based on original data
monthlyvaricoeff_df = pd.DataFrame(monthly_varicoef.items(),columns=['province', 'monthly_varicoeff'])
monthlyvaricoeff_df = monthlyvaricoeff_df.drop([31])
monthlyvaricoeff_df = monthlyvaricoeff_df.set_index('province')
monthlyvaricoeff_df = monthlyvaricoeff_df.reindex(index=data['province'])
monthlyvaricoeff_df = monthlyvaricoeff_df.reset_index()

#converting annual coefficient of variation to dataframe and sorting it based on original data
intanvaricoeff_df = pd.DataFrame(interannual_varicoef.items(),columns=['province', 'rain_coeff_variability'])
intanvaricoeff_df = intanvaricoeff_df.drop([31])
intanvaricoeff_df = intanvaricoeff_df.set_index('province')
intanvaricoeff_df = intanvaricoeff_df.reindex(index=data['province'])
intanvaricoeff_df = intanvaricoeff_df.reset_index()

# adding calculated coefficients of variation to the data
data['rain_coeff_variation']=intanvaricoeff_df['rain_coeff_variability']
data['monthly_varicoeff']= monthlyvaricoeff_df['monthly_varicoeff']

In [5]:
#Creating a dataframe consisting of necessary variables for the calculation of sub-indicators
variables=pd.DataFrame()
variables['province']=data.province
variables['irwr']=data.precipitation-data.evaporation
variables['surf_irwr']=(data.precipitation-data.evaporation)*data.runoff_coeff
variables['withdraw_surf']=data.iloc[:,5:8].sum(axis=1)
variables['gw_irwr']=(data.precipitation-data.evaporation)*(1-data.runoff_coeff)
variables['withdraw_gw']=data.iloc[:,8:11].sum(axis=1)
variables['withdraw_agr']=data.withdraw_gw_agr+data.withdraw_surf_agr
variables['withdraw_ind']=data.withdraw_gw_ind+data.withdraw_surf_ind
variables['access_sanitation_total']=(data.access_sanitation_urban*data.urban_pop_ratio+data.access_sanitation_rural*(1-data.urban_pop_ratio))
variables['deficit_gw_annual']=data.annual_gw_variation*(-1)
variables['deficit_gw_aggregate']=data.aggregate_gw_variation*(-1)
variables['withdraw_total']=np.NaN
variables['withdraw_total']=data[['withdraw_surf_agr','withdraw_surf_ind','withdraw_surf_dom','withdraw_gw_agr','withdraw_gw_ind','withdraw_gw_dom']].sum(axis=1)
variables['agr_withdraw_ratio']=variables['withdraw_agr']/variables['withdraw_total']
variables['access_sanitation_total']=data['access_sanitation_urban']*data['urban_pop_ratio']+data['access_sanitation_rural']*(1-data['urban_pop_ratio'])

#setting negative deficit values equal to zero
variables.loc[variables['deficit_gw_annual'] < 0 ,'deficit_gw_annual']=0
variables.loc[variables['deficit_gw_aggregate'] < 0 ,'deficit_gw_aggregate']=0
variables['withdraw_gw_allowable']=variables.gw_irwr-(variables.deficit_gw_aggregate/20)
variables['withdraw_agr_decrease']=variables.withdraw_gw-variables.withdraw_gw_allowable
variables.loc[variables['withdraw_agr_decrease']<0,'withdraw_agr_decrease']=0
variables['agrwat_lost_ratio']=variables.withdraw_agr_decrease/variables.withdraw_agr

In [6]:
#Creating a dataframe for the sub-indicators
#Dimension 1: Resources
sub_indicators=pd.DataFrame()
sub_indicators['irwr_percap']=variables.irwr*(10**6)/data.population
# sub_indicators['pop_growth']=1+data['pop_growth']/100
# sub_indicators['irwr_percap']=sub_indicators['irwr_percap']/sub_indicators['pop_growth']
sub_indicators['rain_coeff_variation']=data['rain_coeff_variation']
sub_indicators['monthly_varicoeff']=data['monthly_varicoeff']
sub_indicators['anomaly_rain']=abs(data['anomaly_rain'])
sub_indicators['anomaly_temp']=data['anomaly_temp']
sub_indicators['withdraw_surf_ratio']=variables.withdraw_surf/variables.surf_irwr
sub_indicators['withdraw_gw_ratio']=variables.withdraw_gw/variables.gw_irwr
sub_indicators['gw_andef_withdraw']=variables.deficit_gw_annual/variables.withdraw_gw
sub_indicators['gw_agdef_gwrwr']=variables.deficit_gw_aggregate/variables.gw_irwr
sub_indicators['agr_dependency_gw']=data.withdraw_gw_agr/variables.withdraw_agr
sub_indicators['urbanwat_dependency_gw']=data.withdraw_urban_gw/data.produced_urban_wat
sub_indicators['ruralwat_dependency_gw']=data.withdraw_rural_gw/data.produced_rural_wat
sub_indicators['ind_dependency_gw']=data.withdraw_gw_ind/variables.withdraw_ind
#Dimension 2: access
sub_indicators['quality_proxy']=data.urban_fam_treatwat/data.total_urban_fam
sub_indicators['access_wat_urban']=data['access_wat_urban']
sub_indicators['access_wat_rural']=data['access_wat_rural']
sub_indicators['access_sanitation_urban']=data['access_sanitation_urban']
sub_indicators['access_sanitation_rural']=data['access_sanitation_rural']
sub_indicators['treated_municipal_wastewater']=data.waste_facility_cap/data.daily_produced_waste
sub_indicators['under_stress_pop']=data['under_stress_pop']
#Dimension 3:Economy
sub_indicators['efficiency_agr']=data.agr_added_value/variables.withdraw_agr
sub_indicators['modern_irrig']=data.land_irrig_modern/data.land_irrig_tot
sub_indicators['efficiency_ind']=data.ind_added_value/variables.withdraw_ind
sub_indicators['unaccounted_wat_urban']=data['unaccounted_wat_urban']
sub_indicators['unaccounted_wat_rural']=data['unaccounted_wat_rural']
sub_indicators['employment_lost_agr']=variables.agrwat_lost_ratio*data.agr_employment
sub_indicators['damcap_rwr_ratio']=data.dam_cap/variables.surf_irwr
sub_indicators.index=variables['province']

sub_indicators.loc['khuz','damcap_rwr_ratio']=1
sub_indicators

,irwr_percap,rain_coeff_variation,monthly_varicoeff,anomaly_rain,anomaly_temp,withdraw_surf_ratio,withdraw_gw_ratio,gw_andef_withdraw,gw_agdef_gwrwr,agr_dependency_gw,...,access_sanitation_rural,treated_municipal_wastewater,under_stress_pop,efficiency_agr,modern_irrig,efficiency_ind,unaccounted_wat_urban,unaccounted_wat_rural,employment_lost_agr,damcap_rwr_ratio
province,,,,,,,,,,,,,,,,,,,,,
azsh,861.127793,0.167911,0.789813,5.000897,5.256414,0.732273,0.923415,0.041087,0.967437,0.430997,...,0.2,0.386507,0.640612,0.813364,0.189842,87.289034,18.88,36.1,0.000000,1.752232
azgh,1403.563406,0.195106,0.796540,0.194700,7.791266,0.710510,1.657455,0.020779,0.978911,0.406398,...,1.5,0.389288,0.479603,0.481505,0.160564,22.519221,22.54,28.3,5.727620,0.542467
arda,923.914926,0.135676,0.692551,9.740992,7.644496,1.808390,0.670956,0.112430,1.376552,0.155963,...,0.6,0.430248,0.706333,0.670154,0.092103,41.620506,25.40,29.4,0.000000,0.408199
esfe,1016.058135,0.282742,1.186436,11.119641,2.967688,0.803847,1.843314,0.173304,5.435225,0.673997,...,0.8,0.607885,0.224572,0.282064,0.327892,78.001811,16.14,29.6,4.336779,0.545442
albo,419.735290,0.206869,0.987266,12.616949,2.052015,0.418464,2.095116,0.110538,3.744843,0.640933,...,2.0,0.146851,0.726042,0.878002,0.278864,37.255510,22.30,45.9,1.501284,0.799900
ilam,3551.388587,0.317439,1.258290,12.672674,3.556846,0.519724,0.436938,0.000000,0.246187,0.253712,...,0.0,0.431609,0.079606,0.379819,0.372634,3.570169,21.73,30.2,0.000000,0.195158
bush,880.048307,0.483277,1.821725,18.557750,2.298737,1.181926,0.927418,0.137944,0.745432,0.541508,...,0.0,0.118561,0.224810,0.696666,0.396178,195.044482,26.15,34.7,0.000000,1.545451
tehr,101.051905,0.210653,0.965221,13.088950,2.489188,2.332092,4.544267,0.064889,7.707304,0.600733,...,2.4,0.763453,0.183542,0.663583,0.300769,148.856493,23.94,37.5,0.977396,1.678330
chba,7955.881903,0.241602,1.199024,15.349521,7.728830,0.188574,0.433764,0.030993,0.286989,0.419112,...,0.0,0.391572,0.322739,0.365865,0.550997,7.824678,23.74,45.4,0.000000,0.412302


In [22]:
#Normalizing Data
normalized=sub_indicators*0
max_allowable_withdraw=variables['withdraw_gw_allowable']/variables['gw_irwr']
max_allowable_withdraw.index=sub_indicators.index

for col in ind_type.columns:
    i=ind_type.columns.get_loc(col)
    if ind_type.loc[2,col]=='b': # The bigger the better indicators
        normalized.loc[sub_indicators[col]>=ind_type.loc[0,col],col]=1
        normalized.loc[sub_indicators[col]<=ind_type.loc[1,col],col]=0.01
        cond=(sub_indicators[col]>ind_type.loc[1,col])& (sub_indicators[col]<ind_type.loc[0,col])
        normalized.loc[cond,col]=(sub_indicators.loc[cond,col]-ind_type.loc[1,col])/(ind_type.loc[0,col]-ind_type.loc[1,col])
    elif ind_type.loc[2,col]=='l':      # The lower the better indicators
        normalized.loc[sub_indicators[col]<=ind_type.loc[0,col],col]=1
        normalized.loc[sub_indicators[col]>=ind_type.loc[1,col],col]=0.01
        cond=(sub_indicators[col]<ind_type.loc[1,col])& (sub_indicators[col]>ind_type.loc[0,col])
        normalized.loc[cond,col]=(ind_type.loc[1,col]-sub_indicators.loc[cond,col])/(ind_type.loc[1,col]-ind_type.loc[0,col])
    elif ind_type.loc[2,col]=='diff':    # GW withdrawal to rwr ratio normalization
        cond1=(sub_indicators[col] > max_allowable_withdraw)
        normalized.loc[cond1,col]=0.01
        cond2=sub_indicators[col]<0.25
        normalized.loc[cond2,col]=1
        cond3=(sub_indicators[col] < max_allowable_withdraw) & (sub_indicators[col]>0.25)
        normalized.loc[cond3,col]=(max_allowable_withdraw.loc[cond3]-sub_indicators.loc[cond3,col])/(max_allowable_withdraw.loc[cond3]-0.25)    
# normalized.drop('pop_growth',axis='columns',inplace=True)
for col in normalized.columns:
    normalized.loc[normalized[col]<0.01,col]=0.01

In [23]:
# dam capacity to surface rwr ratio normalization    
variation_coeff_agg=pd.Series(normalized[['rain_coeff_variation','monthly_varicoeff']].mean(axis=1),index=variables['province'])
variation_coeff_bins=pd.cut(variation_coeff_agg,4,labels=['Q1','Q2','Q3','Q4'])
ideal_damcap=pd.Series(index=variables['province'],dtype='float64') 
ideal_damcap[variation_coeff_bins=='Q1']=1
ideal_damcap[variation_coeff_bins=='Q2']=0.9
ideal_damcap[variation_coeff_bins=='Q3']=0.8
ideal_damcap[variation_coeff_bins=='Q4']=0.7
normalized['damcap_rwr_ratio']=damcap_normalize(sub_indicators['damcap_rwr_ratio'],ideal_damcap)   
normalized.loc['khuz','damcap_rwr_ratio']=1

# modifying modern irrigation indicator vlues based on aggregate GW reservoir deficits
# gw_deficit_agg=pd.Series(aggregate(normalized[['gw_agdef_gwrwr','gw_andef_withdraw']]),index=variables['province'])
# gw_deficit_bins=pd.qcut(gw_deficit_agg,4,labels=['Q1','Q2','Q3','Q4'])
# gw_modifier=pd.Series(index=variables['province'],dtype='float64')
# gw_modifier[gw_deficit_bins=='Q1']=0.8
# gw_modifier[gw_deficit_bins=='Q2']=0.9
# gw_modifier[gw_deficit_bins=='Q3']=1
# gw_modifier[gw_deficit_bins=='Q4']=1
# normalized['modern_irrig']=normalized['modern_irrig'].multiply(gw_modifier)


In [38]:
a=0.5
indicators=pd.DataFrame(index=normalized.index)
indicators['irwr_percap']=normalized['irwr_percap']
indicators['irwr_climate']=  normalized[['rain_coeff_variation','monthly_varicoeff','anomaly_rain','anomaly_temp']] .mean(axis=1)
indicators['water_stress']=  normalized[['withdraw_surf_ratio','withdraw_gw_ratio']] .mean(axis=1)
indicators['deficit_gw']=  normalized[['gw_andef_withdraw','gw_agdef_gwrwr']] .mean(axis=1)
indicators['dependency_gw']=  normalized[['agr_dependency_gw','ruralwat_dependency_gw','urbanwat_dependency_gw','ind_dependency_gw']] .mean(axis=1)
indicators['quality_proxy']=normalized['quality_proxy']
indicators['access_drinking_water']=  normalized[['access_wat_urban','access_wat_rural']] .mean(axis=1)
indicators['under_stress_pop']=normalized['under_stress_pop']
indicators['sanitation']=  normalized[['access_sanitation_urban','access_sanitation_rural','treated_municipal_wastewater']] .mean(axis=1)
indicators['efficiency_agr']=normalized['efficiency_agr']
indicators['modern_irrig']=normalized['modern_irrig']
indicators['efficiency_ind']=normalized['efficiency_ind']
indicators['unaccounted_water']=  normalized[['unaccounted_wat_urban','unaccounted_wat_rural']] .mean(axis=1)
indicators['employment_lost']=normalized['employment_lost_agr']
indicators['dam_capacity']=normalized['damcap_rwr_ratio']


In [39]:
from scipy.optimize import linprog
weight_df=indicators*0

for i in range(len(indicators.index)):
    c=indicators.iloc[i,:]*-1
    A=indicators
    b=[1]*31
    limits=[(0,None)]*len(indicators.columns)
    weight_df.iloc[i,:]=linprog(c, A_ub=A, b_ub=b  , bounds=limits)['x']
weight_sum=weight_df.sum(axis=1)
weight_df=weight_df.divide(weight_sum,axis=0).round(6)
sub_shares=indicators*weight_df
WSI=sub_shares.sum(axis=1).sort_values()


In [44]:
weight_df_geo=indicators*0
indicators_log=np.log(indicators)
for i in range(len(indicators_log.index)):
    c=indicators_log.iloc[i,:]*-1
    A=indicators_log
    b=[1]*31
    limits=[(0,None)]*len(indicators_log.columns)
    weight_df_geo.iloc[i,:]=linprog(c, A_ub=A, b_ub=b  , bounds=limits)['x']
# weight_sum=weight_df.sum(axis=1)
sub_shares_geo=indicators.pow(weight_df)
# WSI=sub_shares.sum(axis=1).sort_values()
weight_df_geo

,irwr_percap,irwr_climate,water_stress,deficit_gw,dependency_gw,quality_proxy,access_drinking_water,under_stress_pop,sanitation,efficiency_agr,modern_irrig,efficiency_ind,unaccounted_water,employment_lost,dam_capacity
province,,,,,,,,,,,,,,,
azsh,1.367644e-14,2.487442e-13,1.321544e-14,2.621340e-14,6.973949e-14,3.695353e-13,3.244590e-13,1.195283e-10,4.127315e-14,1.255989e-11,7.405300e-15,9.197874e-14,3.526685e-13,2.520278e-01,1.087093e-14
azgh,5.918757e-13,5.688869e-12,6.804012e-13,3.255131e-13,2.535844e-12,3.937830e-11,1.151432e-11,4.127737e-09,4.053500e-12,1.808583e-13,2.909281e-13,3.370760e-13,2.027491e-11,7.299852e-12,1.445756e-12
arda,2.578902e-12,3.137230e-12,1.104550e-11,7.400450e-10,2.863997e-10,2.446062e-10,3.266923e-10,1.030552e-06,5.211811e-11,9.572024e-10,2.793834e-11,1.893117e-11,2.214249e-10,2.620899e-01,3.227188e-10
esfe,2.078330e-10,6.248741e-10,2.159453e-10,1.076995e-09,1.380253e-09,2.496799e-09,8.671578e-11,7.706801e-06,9.000127e-10,3.679873e-10,1.889033e-10,3.536178e-10,3.255545e-10,2.438807e-09,1.193177e-09
albo,1.225300e-13,1.146114e-12,4.643518e-13,1.519805e-12,2.129535e-13,3.959741e-12,1.239101e-12,5.456214e-11,3.245930e-13,3.323601e-01,2.872738e-13,2.133083e-13,5.772778e-13,6.346959e-12,1.968244e-08
ilam,2.636836e-10,4.971751e-10,2.675322e-11,8.185774e-10,3.632975e-10,9.694784e-10,9.026632e-10,4.332293e-06,2.168231e-10,2.066922e-10,2.925854e-10,6.609636e-11,1.603954e-09,2.925814e-01,1.641314e-10
bush,4.316498e-14,3.698152e-14,1.662566e-14,2.403251e-12,7.052247e-13,2.414468e-14,6.052676e-14,2.837914e-09,2.282576e-14,1.993837e-12,3.126601e-13,1.192682e-01,2.532416e-13,3.639017e-01,2.217911e-14
tehr,5.442721e-11,6.987406e-10,1.093538e-11,6.624948e-10,8.854678e-11,7.559791e-10,1.121181e-09,2.510425e-06,3.950672e-10,8.167878e-10,1.530707e-10,4.312052e-12,3.584307e-10,1.498596e-10,3.299856e-10
chba,4.545330e-11,4.933215e-11,2.042105e-11,1.621621e-09,1.655561e-11,8.841257e-11,3.807857e-11,9.305913e-08,5.754838e-11,3.511109e-11,5.581025e-11,1.240233e-11,7.202702e-11,3.094495e-01,1.411082e-10


In [11]:
from scipy.optimize import linprog
weight_d1=indicators.iloc[:,:5]*0
indicators_d1=indicators.iloc[:,:5]
for i in range(len(indicators_d1.index)):
    c=indicators_d1.iloc[i,:]*-1
    A=indicators_d1
    b=[1]*31
    limits=[(0.05,None)]*len(indicators_d1.columns)
    weight_d1.iloc[i,:]=linprog(c, A_ub=A, b_ub=b  , bounds=limits)['x']
weight_sum1=weight_df.sum(axis=1)
weight_d1=weight_d1.divide(weight_sum1,axis=0).round(6)
Resources=(indicators_d1*weight_df).sum(axis=1)

In [12]:
weight_d2=indicators.iloc[:,5:9]*0
indicators_d2=indicators.iloc[:,5:9]
for i in range(len(indicators_d2.index)):
    c=indicators_d2.iloc[i,:]*-1
    A=indicators_d2
    b=[1]*31
    limits=[(0.05,None)]*len(indicators_d2.columns)
    weight_d2.iloc[i,:]=linprog(c, A_ub=A, b_ub=b  , bounds=limits)['x']
weight_sum2=weight_df.sum(axis=1)
weight_d2=weight_d2.divide(weight_sum2,axis=0).round(6)
Access=(indicators_d2*weight_df).sum(axis=1)

In [13]:
weight_d3=indicators.iloc[:,9:]*0
indicators_d3=indicators.iloc[:,9:]
for i in range(len(indicators_d3.index)):
    c=indicators_d3.iloc[i,:]*-1
    A=indicators_d3
    b=[1]*31
    limits=[(0.05,None)]*len(indicators_d3.columns)
    weight_d3.iloc[i,:]=linprog(c, A_ub=A, b_ub=b  , bounds=limits)['x']
weight_sum3=weight_df.sum(axis=1)
weight_d3=weight_d3.divide(weight_sum3,axis=0).round(6)
Economy=(indicators_d3*weight_df).sum(axis=1)
